<a href="https://colab.research.google.com/github/pi-aire/dataminig_sampling/blob/main/TP_DataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Échantillonnage direct de l'espace des motifs

### BRUNEAU Richard - VASLIN Pierre





In [45]:
import numpy as np
import scipy.special as sps
import math
import matplotlib
import pandas as pd
import random

In [46]:
# Le dataframe
df = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
del df[37]
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
0,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
1,1,3,5,7,9,12,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
2,1,3,5,7,9,12,13,16,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74
3,1,3,5,7,9,11,13,15,17,20,21,23,25,27,29,31,34,36,38,40,42,44,47,48,50,52,54,56,58,60,62,64,66,68,70,72,74
4,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,34,36,38,40,42,44,46,48,51,52,54,56,58,60,62,64,66,68,70,72,74


## Question 1

In [73]:
def algoFrequences(df:pd.DataFrame,nb_pattern)-> tuple:
  #R:set = set()
  R = []
  IsInR = set()
  P: dict = dict()
  w = np.zeros(df[0].count())
  totalW = 0
    
  # set les probas
  for i in range(len(w)):
    w[i] = math.pow(2,len(df.iloc[i]))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    random_row = random.uniform(0,totalW)
    # On cherche la ligne
    row = 0
    v = 0
    for i in range(len(w)):
      if  v > random_row:
        row = i - 1
        break
      v += w[i]
    # On selctionne un motif 
    pattern = np.random.choice(
        df.iloc[row],
        random.randint(1, len(df.iloc[row]) - 1 )
    )
    # On ajoute seulement les motifes non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

On calcule le poids total de toutes les lignes dans un premier temps afin de diviser le poids des lignes dans le tableau.

In [75]:
algoFrequences(df,10)

[array([17, 50]),
 array([18, 12, 71, 56, 25, 42, 29, 64, 58, 28,  5, 46, 66, 20, 66, 50, 18,
        46, 18, 58, 58, 42, 36,  3, 40, 64, 34, 60, 14]),
 array([55, 29, 72,  5, 36, 66, 36, 17, 21, 44, 60, 31, 58, 13, 46, 38, 28,
        48]),
 array([62, 60, 64,  9, 39, 44,  5, 20, 40,  9, 14, 58, 25, 46, 44, 14, 62,
        66, 18, 46, 14, 20, 74,  9, 58, 56,  9, 69, 52, 12, 44, 18, 39]),
 array([40, 27,  9, 32, 36, 73, 64, 74,  3, 58,  1, 38, 46, 34]),
 array([71, 46, 27, 52, 52, 31, 74, 58,  5, 39, 34,  5, 69,  7, 48, 36, 45,
         5]),
 array([42, 75, 23, 62, 62, 11, 72, 15, 40, 19, 68,  7, 29, 36, 29, 29, 25,
        48, 19, 15, 19, 64, 75, 22, 64,  5, 29, 72, 50, 34, 15, 72, 56, 19]),
 array([ 1,  5, 69, 31, 66, 69, 60, 52, 29, 40, 15, 29, 26, 47, 11, 64, 20]),
 array([ 9, 31, 60,  9, 64, 29,  9, 62, 21,  3,  2, 66,  7, 58, 60,  5, 36,
        34, 21, 39, 55, 29, 46, 16, 23, 66, 60,  7,  3, 46, 16, 62]),
 array([72, 11, 17, 34, 68, 44, 47, 26, 40, 75,  1, 14, 26, 68, 55, 26])]

## Question 2

In [84]:
def algoArea(df:pd.DataFrame,nb_pattern)-> tuple:
  R = []
  IsInR = set()
  
  # set les probas
  w = np.zeros(df[0].count())
  totalW = 0
  for i in range(1, len(w)):
    w[i] = len(df.iloc[i]) * (2 ** (len(df.iloc[i]) - 1))
    totalW += w[i]
  
  # on selectionne 
  while len(R) < nb_pattern:
    # On cherche la ligne
    random_row = random.uniform(0,totalW)
    v, row = 0,0
    for i in range(len(w)):
      if v > random_row:
        row = i - 1
        break
      v += w[i]
      
    # On set les probabilité de k (taille du motif)
    # On veut que le sous ensemble avec une probabilté 
    # proportinelle à la taille de la datarecord (ligne)
    ks = np.zeros(len(df.iloc[row]))
    totalK = 0
    for i in range(len(df.iloc[row])):
        ks[i] = sps.binom(len(df.iloc[row]), i + 1)
        totalK += ks[i]
    random_kp = random.uniform(0, totalK)
    # On cherche la ligne
    k, v = 0,0
    for i in range(len(df.iloc[row])):
      if v > random_kp:
        k = i - 1
        break
      v += ks[i]
    
    pattern = np.random.choice(
        df.iloc[row],
        k
    )
    
    # On ajoute seulement les motifs non présent dans l'ensemble R
    IsInR.add(np.array2string(pattern))
    if len(IsInR) != len(R):
      R.append(pattern)
  return R

In [86]:
algoArea(df,10)

[array([44, 46,  2, 64, 72, 56, 60, 54, 74, 25, 72, 33, 70, 34,  3, 62, 48,
         9, 25]),
 array([31, 69, 12, 20, 12, 56, 36, 50, 55, 29, 40, 42,  9, 25, 40, 62, 58,
        56]),
 array([55, 66, 51, 70, 66, 56, 44, 64, 38,  2, 23, 36, 58, 42, 56, 66, 66,
        23, 48, 72]),
 array([ 1, 36, 50, 60, 14, 62, 23, 22, 45, 29,  9, 17, 11, 38, 28, 58, 22,
         1, 25, 52, 15, 52]),
 array([ 3, 23, 31, 36, 66, 75, 52, 40, 21, 42, 14,  3, 40, 34, 48, 60, 64,
        48, 58,  1]),
 array([23, 42, 25, 23, 74, 62,  3, 72, 52,  2, 28,  9, 62,  5, 31, 36, 16,
        16]),
 array([ 9, 19, 19, 44, 42, 54, 44, 46, 46,  7, 24, 74,  5, 38,  7, 56, 16,
        50,  7,  2]),
 array([ 4, 52, 12,  4, 66, 24, 29, 21, 48, 25, 25,  9, 21, 36, 58, 38, 66,
        62, 27, 20, 31, 38, 15, 62]),
 array([68, 68, 52, 17, 44,  5, 12, 36, 29, 17,  7, 68, 12, 17,  3, 74, 31,
        34, 60, 28,  5, 31, 70, 68, 60]),
 array([71, 48, 10, 74, 24, 36, 19, 58, 71, 58, 74, 54, 46, 69, 32,  3, 14,
        14])]

##Question 3

In [117]:
df.iloc[0][5]

11

In [97]:
def frequences(df, patterns):
  frequencesP = np.zeros(len(patterns))
  for i in range(df.count()[0]):
    indexs = np.zeros(len(patterns))
    for j in range(df.iloc[j].count()):
      for ip in range(len(patterns)):
        if len(patterns[ip])!=indexs[ip] and df.iloc[i][j] == patterns[ip][indexs[ip]]:
          indexs[ip] += 1
          if len(patterns[ip])== indexs[ip]:
            frequencesP[ip] += 1
      


In [98]:
frequence(algoFrequences(df,10),df)

TypeError: ignored

## Question 4

In [89]:
dataset_1 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/chess.txt", sep=" ",header=None)
#dataset_2 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/kdd.txt", sep=" ",header=None,)
dataset_3 = pd.read_table("https://bitbucket.org/anesbendimerad/sigibbssamplingcode/raw/6699a50508fe177ee0c00dcc7d8e5390ee53688a/ItemsetDatasets/mushrooms.txt", sep=" ",header=None)
#dataset_4 = pd.read_table("http://www.philippe-fournier-viger.com/spmf/datasets/BMS1_spmf", sep=" ",header=None)